<a href="https://colab.research.google.com/github/adarshtomar333/YouTube-Parser/blob/main/YTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install yt-dlp
!pip install -U openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 15.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s

In [1]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
import os
import time
import yt_dlp
import subprocess
import whisper
import re

# Path to store the downloaded and processed audio
OUTPUT_DIR = "/content/YT"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Function to sanitize filenames (remove invalid characters)
def sanitize_filename(title):
    return re.sub(r'[\\/*?:"<>|]', "_", title)

# Function to get video title
def get_video_title(video_url):
    ydl_opts = {"quiet": True, "no_warnings": True, "skip_download": True, "extract_flat": True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(video_url, download=False)
        return sanitize_filename(info["title"]) if "title" in info else "video_audio"

# Function to download audio from YouTube
def download_audio(video_url):
    title = get_video_title(video_url)
    audio_path = os.path.join(OUTPUT_DIR, f"{title}.mp3")

    ydl_opts = {
        "format": "bestaudio/best",
        "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "mp3", "preferredquality": "192"}],
        "outtmpl": os.path.join(OUTPUT_DIR, f"{title}.%(ext)s"),  # Store using title
    }

    try:
        print(f"\n📥 Downloading audio for '{title}'...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])

        time.sleep(2)  # Give time for the file to save

        # Check if the file exists
        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"❌ MP3 file not found after download!")

        print(f"✅ Audio downloaded successfully at {audio_path}.")
        return title  # Return the title for further steps
    except Exception as e:
        print(f"❌ Error downloading audio: {e}")
        exit()

# Function to convert MP3 to WAV
def convert_to_wav(title):
    mp3_path = os.path.join(OUTPUT_DIR, f"{title}.mp3")
    wav_path = os.path.join(OUTPUT_DIR, f"{title}.wav")

    if not os.path.exists(mp3_path):
        print("❌ Error: MP3 file not found!")
        exit()

    print(f"\n🎵 Converting '{title}.mp3' to WAV...")
    command = [
        "ffmpeg", "-i", mp3_path, "-ar", "16000", "-ac", "1",
        "-c:a", "pcm_s16le", wav_path, "-y"
    ]

    try:
        subprocess.run(command, check=True)
        print(f"✅ Audio converted to WAV: {wav_path}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Error converting MP3 to WAV: {e}")
        exit()

# Function to transcribe audio using Whisper in English
def transcribe_audio(title):
    """Transcribe audio using Whisper with automatic language detection."""
    wav_path = os.path.join(OUTPUT_DIR, f"{title}.wav")
    transcript_file = os.path.join(OUTPUT_DIR, f"{title}.txt")

    if not os.path.exists(wav_path):
        print("❌ Error: WAV file not found!")
        exit()

    print(f"\n📝 Transcribing '{title}.wav' with automatic language detection...")
    model = whisper.load_model("large")  # Use "small" or "large" for better accuracy
    result = model.transcribe(wav_path)  # No language specified for automatic detection

    transcript = result["text"]
    print("\n✅ Transcript Ready!\n")

    # Save transcript to a file
    with open(transcript_file, "w", encoding="utf-8") as f:
        f.write(transcript)

    print(f"📄 Transcript saved to: {transcript_file}")

# Function to clean up temporary files
def cleanup_files(title):
    mp3_path = os.path.join(OUTPUT_DIR, f"{title}.mp3")
    wav_path = os.path.join(OUTPUT_DIR, f"{title}.wav")

    try:
        if os.path.exists(mp3_path):
            os.remove(mp3_path)
            print(f"🗑️ Deleted temporary file: {mp3_path}")
        if os.path.exists(wav_path):
            os.remove(wav_path)
            print(f"🗑️ Deleted temporary file: {wav_path}")
    except Exception as e:
        print(f"❌ Error cleaning up files: {e}")

# Main execution
video_url = input("\n🎥 Enter YouTube video URL: ").strip()
title = download_audio(video_url)
convert_to_wav(title)
transcribe_audio(title)
cleanup_files(title)  # Clean up temporary files


🎥 Enter YouTube video URL: https://www.youtube.com/watch?v=yqL6ss0uR5o&t=1s

📥 Downloading audio for 'The Geography of Uttarakhand'...
[youtube] Extracting URL: https://www.youtube.com/watch?v=yqL6ss0uR5o&t=1s
[youtube] yqL6ss0uR5o: Downloading webpage
[youtube] yqL6ss0uR5o: Downloading tv client config
[youtube] yqL6ss0uR5o: Downloading player 5ae7d525
[youtube] yqL6ss0uR5o: Downloading tv player API JSON
[youtube] yqL6ss0uR5o: Downloading ios player API JSON
[youtube] yqL6ss0uR5o: Downloading m3u8 information
[info] yqL6ss0uR5o: Downloading 1 format(s): 251
[download] Destination: /content/YT/The Geography of Uttarakhand.webm
[download] 100% of   18.34MiB in 00:00:00 at 25.97MiB/s  
[ExtractAudio] Destination: /content/YT/The Geography of Uttarakhand.mp3
Deleting original file /content/YT/The Geography of Uttarakhand.webm (pass -k to keep)
✅ Audio downloaded successfully at /content/YT/The Geography of Uttarakhand.mp3.

🎵 Converting 'The Geography of Uttarakhand.mp3' to WAV...
✅ Aud

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
